In [3]:
import numpy as np
import pennylane as qml
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.linalg import eigh
import pandas as pd
from itertools import product
import math
from scipy.linalg import eigh

In [6]:
# example matrix from PennyLane
a = 0.25
b = 0.75

A = np.array(
    [[a,  0, 0,  b],
     [0, -a, b,  0],
     [0,  b, a,  0],
     [b,  0, 0, -a]])

val, vec = eigh(A)
print('eigenvalues : ', val)
print('eigenvectors : \n', vec)

eigenvalues :  [-0.79056942 -0.79056942  0.79056942  0.79056942]
eigenvectors : 
 [[ 0.58471028  0.          0.81124219  0.        ]
 [ 0.         -0.81124219  0.          0.58471028]
 [ 0.          0.58471028  0.          0.81124219]
 [-0.81124219  0.          0.58471028  0.        ]]


In [56]:
def lcu(A):
    LCU = qml.pauli_decompose(A)
    coeff, op = LCU.terms()
    norm_factor = sum(abs(coeff))
    norm_coeff = (np.sqrt(coeff) / np.linalg.norm(np.sqrt(coeff)))
    return norm_coeff, op, norm_factor

def lcu_reduction(coeff, op, N): # extract N top terms
    idx = np.argsort(coeff)[-N:][::-1]
    coeff_red = [coeff[i] for i in idx]
    op_red = [op[i] for i in idx]
    norm_factor = sum(abs(op_red))
    norm_coeff_red = (np.sqrt(op_red) / np.linalg.norm(np.sqrt(op_red)))
    return norm_coeff_red, op_red, norm_factor

def init_config(A, coeff_red, op_red, norm_factor):
    n_control = int(math.log2(len(coeff_red)))
    n_target = int(math.log2(A.shape[0]))
    return n_control, n_target

def get_basis_state(n_qubits):
    return [list(state) for state in product([0, 1], repeat=n_qubits)]

def init_state(iter_num, input_coeff, n_target):
    if iter_num==0:
        for i in reg['target']:
            qml.Hadamard(i)
    else:
        base = get_basis_state(n_target)
        qml.Superposition(input_coeff, base, reg['target'], work_wire=reg['super_work'])

def block_encoding(coeff, op):
    qml.StatePrep(coeff, wires=reg['control'])
    qml.Select(op, control=reg['control'])
    qml.adjoint(qml.StatePrep(coeff, wires=reg['control']))

def zero_projector(n_qubits, measured_wires, outcomes):
    return [
        i for i in range(2 ** n_qubits)
        if all(format(i, f'0{n_qubits}b')[wire] == str(outcome) 
               for wire, outcome in zip(measured_wires, outcomes))]

def get_next_vec(vec, reg):
    proj_idx = zero_projector(n_tot, measured_wires=reg.tolist(), outcomes=np.zeros(len(reg), dtype=int).tolist())
    proj = np.array([vec[i] for i in proj_idx]) # project onto |0> in the control qubit
    proj = proj[0::2] # effectively remove the auxiliary qubit
    proj_norm_coeff = np.linalg.norm(proj)
    proj_norm = proj / proj_norm_coeff
    # iter_num +=1
    return proj_norm_coeff, np.real(proj), np.real(proj_norm)

In [64]:
coeff, op, norm_factor = lcu(A)
n_control, n_target = init_config(A, coeff, op, norm_factor)
n_tot = wires=n_control+n_target+1
scaling_factor = sum(abs(coeff))

dev = qml.device("default.qubit", n_tot) 
reg = qml.registers({"control": n_control, "target": n_target, "super_work":1}) 
op = [qml.map_wires(op_elem, {i: i + n_control for i in range(n_target)}) for op_elem in op]

@qml.qnode(dev)
def power_circuit(A, iter_num, coeff, op, input_coeff, n_target):
    init_state(iter_num, input_coeff, n_target)
    block_encoding(coeff, op)
    return qml.expval(qml.Hermitian(A, wires=reg['target'])), qml.state()

In [98]:
def power_method(A, iter_target=5):
    iter_num = 0
    input_coeff = 0
    proj_norm_coeffs = []
    vals = []
    vecs = []
    
    while(iter_num<iter_target):
        val, vec = power_circuit(A, iter_num, coeff, op, input_coeff, n_target)
        proj_norm_coeff, proj, proj_norm = get_next_vec(vec, reg['control'])
        
        if iter_num!=1:
            proj_norm_coeffs.append(proj_norm_coeff)
            
        vec = np.prod(proj_norm_coeffs) * proj
        vecs.append(vec)
        vals.append((vec.T @ (A @ vec)) / (vec.T @ vec)) # rayleigh coeff
        
        # updates for the next iteration
        iter_num += 1
        input_coeff = proj_norm
    return vals[-1], vecs[-1]

In [107]:
power_method(A,2)

(0.75, array([0.3125, 0.3125, 0.3125, 0.3125]))